### Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import urllib.parse 

### Put all together

In [2]:
# create dataframe (in the beginning it is empty)
df_restaurant = pd.DataFrame(columns=['Restaurant Name', 'Address', 'Phone', 'Email', 'Homepage', 'Info'])

# sequence of numbers - 1 to 3 (4 is not included)
for i in range(1,4):

    # varibale for pages withing website
    website = 'https://www.yellowpages.com/search?search_terms=restaurant&geo_location_terms=New%20York%2C%20NY&page=' + str(i)

    # request
    response = requests.get(website)
    
    # create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # result container
    result_container = soup.find_all('div', {'class':'result'})
    
    # URL Part 1
    url_part_1 = 'https://www.yellowpages.com/'
    
    url_part_2 = []

    # loop through results
    for item in result_container:
        
    # loop through links
        for link in item.find_all('a', {'class': 'business-name'}):
            url_part_2.append(link.get('href'))
    
    # Join URL 1 and URL 2 in order to get the abosolute URL
    url_joined = []

    for link_2 in url_part_2:
        url_joined.append(urllib.parse.urljoin(url_part_1, link_2))
        
        
    # loop through all joined links
    for link in url_joined:
        response = requests.get(link)

        # create soup object
        soup = BeautifulSoup(response.content, 'html.parser')

        # name
        try:
            name = soup.find('h1').get_text()
        except:
            name = 'n/a'

        # address 
        try:
            address = soup.find('h2', {'class': 'address'}).get_text()
        except:
            address = 'n/a'

        # phone
        try:
            phone = soup.find('p', {'class':'phone'}).get_text()
        except:
            phone = 'n/a'

        # email
        try:
            email = soup.find('a', {'class': 'email-business'}).get('href').split('mailto:')[1]
        except:
            email = 'n/a'

        # website
        try:
            website = soup.find('a', {'class': 'website-link'}).get('href')
        except:
            website = 'n/a'

        # general info
        try:
            info = soup.find('dd', {'class': 'general-info'}).get_text()
        except:
            info = 'n/a'
            
        # Pandas Dataframe
        df_restaurant = df_restaurant.append({'Restaurant Name': name, 'Address':address, 'Phone': phone,
                                             'Email':email, 'Homepage': website, 'Info': info}, ignore_index=True)
    


### Output Pandas Dataframe

In [3]:
df_restaurant

,Restaurant Name,Address,Phone,Email,Homepage,Info
0,Mr. K's,n/a,Phone: (212) 583-1668,info@movingmaninc.com,http://www.mrksny.com,We offer exclusive seating to you and your par...
1,Seven's Turkish Grill,n/a,Phone: (212) 724-4700,n/a,n/a,Here at Seven's Mediterranean Turkish Grill we...
2,Todaro Brothers,n/a,Phone: (212) 532-0633,eat@todarobros.com,http://www.todarobros.com,NYC's Neighborhood Market since 1917. Todaro ...
3,Babbo,n/a,Phone: (212) 777-0303,hospitality@babbonyc.com,http://www.babbonyc.com,Babbo Ristorante e Enoteca is an exuberant cel...
4,Sparks Steak House,n/a,Phone: (212) 687-4855,office@sparkssteakhouse.com,http://www.sparkssteakhouse.com,"Established in 1966, Sparks Steak House featur..."
...,...,...,...,...,...,...
85,TGI Fridays,n/a,Phone: (646) 556-6381,n/a,http://www.tgifridays.com,n/a
86,Cosi,n/a,Phone: (212) 588-1225,contactus@getcosi.com,http://www.getcosi.com,n/a
87,Barfly Sports Bar & Restaurant,n/a,Phone: (212) 473-9660,n/a,http://www.barflyny.com,n/a
88,Desnuda,n/a,Phone: (212) 254-3515,peter@desnudany.com,http://www.desnudany.com/about.php,n/a


### Store in Excel

In [4]:
df_restaurant.to_excel('results_multiple_pages.xlsx', index=False)